<a href="https://colab.research.google.com/github/Champioa00/LAB1/blob/master/Lab1Workingsession.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Anthony Champion ##
## IST 718 - LAB 1 ##
## 1/30/2022 ##
## Question: How can we recommend the best salary for our next head football coach? ##

In [2]:
## Load Required Packages ##
import pandas as pd
import io
import numpy as np 
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm
import statsmodels.formula.api as smf

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
## Load Coach Data Frame ##
coachdf = pd.read_csv('https://raw.githubusercontent.com/Champioa00/LAB1/master/Coaches9.csv')


In [4]:
## Explore Coach Data Frame ## 
print(coachdf.shape)
print(coachdf.info())
coachdf.head()

(129, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   School        129 non-null    object
 1   Conference    129 non-null    object
 2   Coach         129 non-null    object
 3   SchoolPay     129 non-null    object
 4   TotalPay      129 non-null    object
 5   Bonus         129 non-null    object
 6   BonusPaid     129 non-null    object
 7   AssistantPay  129 non-null    object
 8   Buyout        129 non-null    object
dtypes: object(9)
memory usage: 9.2+ KB
None


,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
0,Air Force,Mt. West,Troy Calhoun,885000,885000,247000,--,$0,--
1,Akron,MAC,Terry Bowden,"$411,000","$412,500","$225,000","$50,000",$0,"$688,500"
2,Alabama,SEC,Nick Saban,"$8,307,000","$8,307,000","$1,100,000","$500,000",$0,"$33,600,000"
3,Alabama at Birmingham,C-USA,Bill Clark,"$900,000","$900,000","$950,000","$165,471",$0,"$3,847,500"
4,Appalachian State,Sun Belt,Scott Satterfield,"$712,500","$712,500","$295,000","$145,000",$0,"$2,160,417"


The starting point of this analysis is the coaching dataset.  The first pass data exploration shows there are 9 columns and 129 unqiue observations.  Looking deeper, it's clear the numeric values all have a dollar sign ($) in front of them.  There also looks to be some values missing in BonusPaid and Buyout.  We will need to add some additional data points as well as perform some data transformation and data cleansing.

In [5]:
## Look at one conference and determine what variables need to be cleaned ##
coachdf[coachdf['Conference']=='SEC']

,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
2,Alabama,SEC,Nick Saban,"$8,307,000","$8,307,000","$1,100,000","$500,000",$0,"$33,600,000"
7,Arkansas,SEC,Chad Morris,"$3,500,000","$3,500,000","$1,000,000",--,$0,"$12,500,000"
10,Auburn,SEC,Gus Malzahn,"$6,700,000","$6,705,656","$1,400,000","$375,000",$0,"$32,143,750"
31,Florida,SEC,Dan Mullen,"$6,070,000","$6,070,000","$925,000",--,$0,"$12,000,000"
36,Georgia,SEC,Kirby Smart,"$6,603,600","$6,603,600","$1,150,000","$1,350,000",$0,"$27,917,500"
49,Kentucky,SEC,Mark Stoops,"$4,000,000","$4,013,600","$3,100,000","$300,000",$0,"$15,625,000"
55,LSU,SEC,Ed Orgeron,"$3,500,000","$3,500,000","$1,575,000","$100,000",$0,"$5,291,667"
66,Mississippi,SEC,Matt Luke,"$3,000,000","$3,000,000","$1,550,000",--,$0,$0
67,Mississippi State,SEC,Joe Moorhead,"$2,600,000","$2,600,000","$1,350,000",--,$0,$0
68,Missouri,SEC,Barry Odom,"$2,350,000","$2,350,000","$1,025,000","$375,000",$0,"$1,912,500"


All the dollar signs and commas need to be removed as well as the "--" data points converted to 0. Once completed we will start to introduce addtional datasets into the analysis.

In [6]:
## Select all the numeric columns ## 
num_columns = ['SchoolPay', 'TotalPay', 'Bonus', 'BonusPaid', 'AssistantPay', 'Buyout']

## Remove $, ",", and replace "--" with "0" ## 
for i in num_columns:
    coachdf[i] = coachdf[i].replace('[$,]', '', regex=True)
    coachdf[i] = coachdf[i].replace('[--]', '0', regex=True)
    coachdf[i] = coachdf[i].astype(float)

## Check Coach Dataset ## 
coachdf[coachdf['Conference']=='SEC']

,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
2,Alabama,SEC,Nick Saban,8307000.0,8307000.0,1100000.0,500000.0,0.0,33600000.0
7,Arkansas,SEC,Chad Morris,3500000.0,3500000.0,1000000.0,0.0,0.0,12500000.0
10,Auburn,SEC,Gus Malzahn,6700000.0,6705656.0,1400000.0,375000.0,0.0,32143750.0
31,Florida,SEC,Dan Mullen,6070000.0,6070000.0,925000.0,0.0,0.0,12000000.0
36,Georgia,SEC,Kirby Smart,6603600.0,6603600.0,1150000.0,1350000.0,0.0,27917500.0
49,Kentucky,SEC,Mark Stoops,4000000.0,4013600.0,3100000.0,300000.0,0.0,15625000.0
55,LSU,SEC,Ed Orgeron,3500000.0,3500000.0,1575000.0,100000.0,0.0,5291667.0
66,Mississippi,SEC,Matt Luke,3000000.0,3000000.0,1550000.0,0.0,0.0,0.0
67,Mississippi State,SEC,Joe Moorhead,2600000.0,2600000.0,1350000.0,0.0,0.0,0.0
68,Missouri,SEC,Barry Odom,2350000.0,2350000.0,1025000.0,375000.0,0.0,1912500.0


Add in addtional data sets to help with the analysis

In [7]:
## Load Records and Points Metrics by team and conference ##
recordsdf = pd.read_csv('https://raw.githubusercontent.com/Champioa00/LAB1/master/records.csv')

In [8]:
## Explore Coach Data Frame ## 
print(recordsdf.shape)
print(recordsdf.info())
recordsdf.head()

(130, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SCHOOL                     130 non-null    object 
 1   CONFERENCE                 130 non-null    object 
 2   W                          130 non-null    int64  
 3   L                          130 non-null    int64  
 4   PF                         130 non-null    int64  
 5   PA                         130 non-null    int64  
 6   AVG_POINTS_PER _GAME       130 non-null    float64
 7   DEF_ AVG_POINTS_PER _GAME  130 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 8.2+ KB
None


,SCHOOL,CONFERENCE,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME
0,Cincinnati,AAC,13,1,516,236,36.9,16.9
1,Houston,AAC,12,2,508,286,36.3,20.4
2,UCF,AAC,9,4,478,333,36.8,25.6
3,SMU,AAC,8,4,461,341,38.4,28.4
4,East Carolina,AAC,7,5,356,315,29.7,26.3


In [9]:
## Clean up Numeric Columns ## 
## Select all the numeric columns ## 
recordssdf_num_columns = ['W', 'L', 'PF', 'PA']

## Remove $, ",", and replace "--" with "0" ## 
for i in recordssdf_num_columns:
    recordsdf[i] = recordsdf[i].astype(float)

## Check to see if the number format changed to float ##
print(recordsdf.shape)
print(recordsdf.info())

(130, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SCHOOL                     130 non-null    object 
 1   CONFERENCE                 130 non-null    object 
 2   W                          130 non-null    float64
 3   L                          130 non-null    float64
 4   PF                         130 non-null    float64
 5   PA                         130 non-null    float64
 6   AVG_POINTS_PER _GAME       130 non-null    float64
 7   DEF_ AVG_POINTS_PER _GAME  130 non-null    float64
dtypes: float64(6), object(2)
memory usage: 8.2+ KB
None


In [10]:
## Clean the Conference Data in the Coaches and Records ##

print(len(coachdf.Conference.unique()), coachdf.Conference.unique())
print(len(recordsdf.CONFERENCE.unique()), recordsdf.CONFERENCE.unique())

11 ['Mt. West' 'MAC' 'SEC' 'C-USA' 'Sun Belt' 'Pac-12' 'Ind.' 'Big 12' 'ACC'
 'AAC' 'Big Ten']
11 ['AAC' 'ACC' 'Big 12' 'Big Ten' 'C-USA' 'FBS Independent' 'MAC'
 'Mountain West' 'Pac-12' 'SEC' 'Sun Belt']


It looks like both the coach and record file have the same amount of conferences, lets look deeper and make sure they are fully aligned.

In [11]:
## Align the Conference Data ##

recordsdf_names = ['AAC','ACC','Big 12' ,'Big Ten' ,'C-USA', 'FBS Independent' ,'MAC' ,'Mountain West', 'Pac-12' ,'SEC' ,'Sun Belt']
new_name =        ['AAC' ,'ACC' ,'Big 12', 'Big Ten', 'C-USA' ,'Ind.', 'MAC', 'Mt. West' ,'Pac-12', 'SEC' ,'Sun Belt']  

recordsdf['CONFERENCE']=recordsdf['CONFERENCE'].replace(recordsdf_names, new_name, regex=False)


In [12]:
recordsdf_names

['AAC',
 'ACC',
 'Big 12',
 'Big Ten',
 'C-USA',
 'FBS Independent',
 'MAC',
 'Mountain West',
 'Pac-12',
 'SEC',
 'Sun Belt']

In [13]:
## Check to see if Conference change worked ## 
recordsdf[recordsdf['CONFERENCE']=='Ind.']


,SCHOOL,CONFERENCE,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME
63,Notre Dame,Ind.,11.0,2.0,458.0,256.0,35.2,19.7
64,BYU,Ind.,10.0,3.0,430.0,322.0,33.1,24.8
65,Army West Point,Ind.,9.0,4.0,427.0,290.0,32.8,22.3
66,Liberty,Ind.,8.0,5.0,437.0,280.0,33.6,21.5
67,New Mexico St.,Ind.,2.0,10.0,269.0,485.0,22.4,40.4
68,Massachusetts,Ind.,1.0,11.0,196.0,517.0,16.3,43.1
69,UConn,Ind.,1.0,11.0,187.0,462.0,15.6,38.5



Looks like Conference for both the coaches and records data sets are the same, now lets load in some data set to help find our answer.  Next up is stadium infomration

In [14]:
## Load and Explore Stadium Dataset ##
stadiumdf = pd.read_csv('https://raw.githubusercontent.com/Champioa00/LAB1/master/Stadium.csv')

## Explore Stadium Data Frame ## 
print(stadiumdf.shape)
print(stadiumdf.info())
stadiumdf.head()

(131, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Stadium     131 non-null    object
 1   College     131 non-null    object
 2   Conference  131 non-null    object
 3   Capacity    131 non-null    int64 
 4   Opened      131 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 5.2+ KB
None


,Stadium,College,Conference,Capacity,Opened
0,Michigan Stadium,Michigan,Big Ten,107601,1927
1,Beaver Stadium,Penn State,Big Ten,106572,1960
2,Ohio Stadium,Ohio State,Big Ten,104944,1922
3,Kyle Field,Texas A&M,SEC,102733,1904
4,Neyland Stadium,Tennessee,SEC,102521,1921


The Stadium dataset looks to have the needed School and Conference fields, as well as 131 records.  Let's take a deeper dive into the Conference Names and align those to our coaches and records datasets.

In [15]:
## Clean the data and change numeric fields ##
## Clean up Numeric Columns ## 
## Select all the numeric columns ## 
stadiumdf_num_columns = ['Capacity']

# Change to numeric ## 
for i in stadiumdf_num_columns:
    stadiumdf[i] = stadiumdf[i].astype(float)

## Check to see if the number format changed to float ##
print(stadiumdf.shape)
print(stadiumdf.info())

(131, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Stadium     131 non-null    object 
 1   College     131 non-null    object 
 2   Conference  131 non-null    object 
 3   Capacity    131 non-null    float64
 4   Opened      131 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 5.2+ KB
None


In [16]:
## Align the Conference with Coaches and Records Dataframe ## 
print(len(coachdf.Conference.unique()), coachdf.Conference.unique())
print(len(recordsdf.CONFERENCE.unique()), recordsdf.CONFERENCE.unique())
print(len(stadiumdf.Conference.unique()), stadiumdf.Conference.unique())

11 ['Mt. West' 'MAC' 'SEC' 'C-USA' 'Sun Belt' 'Pac-12' 'Ind.' 'Big 12' 'ACC'
 'AAC' 'Big Ten']
11 ['AAC' 'ACC' 'Big 12' 'Big Ten' 'C-USA' 'Ind.' 'MAC' 'Mt. West' 'Pac-12'
 'SEC' 'Sun Belt']
11 ['Big Ten' 'SEC' 'Big 12' 'Pac 12' 'ACC' 'Independent' 'CUSA'
 'Mountain West' 'MAC' 'AAC' 'Sun Belt']


In [17]:
## All three have 11 unique Conferences, just need to rename a couple conferences ##

stadiumdf_names = ['AAC' ,'ACC' ,'Big 12', 'Big Ten', 'CUSA', 'Independent', 'MAC', 'Mountain West', 'Pac 12', 'SEC', 'Sun Belt']

stadiumdf['Conference']=stadiumdf['Conference'].replace(stadiumdf_names, new_name, regex=False)

print(len(coachdf.Conference.unique()), coachdf.Conference.unique())
print(len(recordsdf.CONFERENCE.unique()), recordsdf.CONFERENCE.unique())
print(len(stadiumdf.Conference.unique()), stadiumdf.Conference.unique())

11 ['Mt. West' 'MAC' 'SEC' 'C-USA' 'Sun Belt' 'Pac-12' 'Ind.' 'Big 12' 'ACC'
 'AAC' 'Big Ten']
11 ['AAC' 'ACC' 'Big 12' 'Big Ten' 'C-USA' 'Ind.' 'MAC' 'Mt. West' 'Pac-12'
 'SEC' 'Sun Belt']
11 ['Big Ten' 'SEC' 'Big 12' 'Pac-12' 'ACC' 'Ind.' 'C-USA' 'Mt. West' 'MAC'
 'AAC' 'Sun Belt']


Time to add the final dataset, Graduation meterics

In [18]:
## Load and Explore Graduation Dataset ##
graddf = pd.read_csv('https://raw.githubusercontent.com/Champioa00/LAB1/master/GradRate.csv')

## Explore Graduation Data Frame ## 
print(graddf.shape)
print(graddf.info())
graddf.head()

(256, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Cohort Year  256 non-null    int64 
 1   School       256 non-null    object
 2   Conference   256 non-null    object
 3   Sport        256 non-null    object
 4   State        256 non-null    object
 5   GSR          256 non-null    int64 
 6   FGR          256 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 14.1+ KB
None


,Cohort Year,School,Conference,Sport,State,GSR,FGR
0,2014,Abilene Christian University,Southland Conference,Football,TX,75,51
1,2014,University of Akron,Mid-American Conference,Football,OH,79,76
2,2014,Alabama A&M University,Southwestern Athletic Conf.,Football,AL,62,54
3,2014,Alabama State University,Southwestern Athletic Conf.,Football,AL,71,42
4,2014,University of Alabama,Southeastern Conference,Football,AL,86,65


In [19]:
## Clean the data and change numeric fields ##
## Clean up Numeric Columns ## 
## Select all the numeric columns ## 
graddf_num_columns = ['GSR' , 'FGR']

# Change to numeric ## 
for i in graddf_num_columns:
    graddf[i] = graddf[i].astype(float)

## Check to see if the number format changed to float ##
print(graddf.shape)
print(graddf.info())

(256, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Cohort Year  256 non-null    int64  
 1   School       256 non-null    object 
 2   Conference   256 non-null    object 
 3   Sport        256 non-null    object 
 4   State        256 non-null    object 
 5   GSR          256 non-null    float64
 6   FGR          256 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 14.1+ KB
None


In [20]:
## Align the Conference with Coaches and Records Dataframe ## 
print(len(graddf.Conference.unique()), graddf.Conference.unique())
new_name

25 ['Southland Conference' 'Mid-American Conference'
 'Southwestern Athletic Conf.' 'Southeastern Conference' 'Conference USA'
 'Colonial Athletic Association' 'Sun Belt Conference' 'Pac-12 Conference'
 'Ohio Valley Conference' 'Big South Conference' 'Big 12 Conference'
 'Mid-Eastern Athletic Conf.' 'Mountain West Conference'
 'Atlantic Coast Conference' 'Independent' 'The Ivy League'
 'Northeast Conference' 'Patriot League' 'Pioneer Football League'
 'Big Sky Conference' 'American Athletic Conference' 'Southern Conference'
 'Missouri Valley Football Conference' 'Big Ten Conference'
 'Horizon League']


['AAC',
 'ACC',
 'Big 12',
 'Big Ten',
 'C-USA',
 'Ind.',
 'MAC',
 'Mt. West',
 'Pac-12',
 'SEC',
 'Sun Belt']

In [21]:
## Transform Conference Column ##

grad_old_name = ['Southland Conference' ,'Mid-American Conference' ,'Southwestern Athletic Conf.' ,'Southeastern Conference' ,'Conference USA' ,'Colonial Athletic Association' ,
'Sun Belt Conference' ,'Pac-12 Conference' ,'Ohio Valley Conference' ,'Big South Conference' ,'Big 12 Conference' ,'Mid-Eastern Athletic Conf.' ,'Mountain West Conference' 
,'Atlantic Coast Conference' ,'Independent' ,'The Ivy League' ,'Northeast Conference' ,'Patriot League' ,'Pioneer Football League' ,'Big Sky Conference' ,'American Athletic Conference'
,'Southern Conference', 'Missouri Valley Football Conference' ,'Big Ten Conference', 'Horizon League']

grad_new_name = ['SLC', 'MAC', 'SWAC', 'SEC', 'C-USA', 'CAA', 'Sun Belt', 'Pac-12', 'OVC', 'Big South', 'Big 12', 'MEAC', 'Mt. West', 'ACC', 'Ind.','IVY', 'NEC', 'PAT', 'PFL', 
                 'Big Sky', 'AAC', 'SOCON', 'MVFC', 'Big Ten' , 'HOL']


graddf['Conference'] = graddf['Conference'].replace(grad_old_name, grad_new_name, regex=False)

## Check to see if Conference change worked ## 
graddf[graddf['Conference']=='IVY']

,Cohort Year,School,Conference,Sport,State,GSR,FGR
23,2014,Brown University,IVY,Football,RI,95.0,0.0
45,2014,Columbia University-Barnard College,IVY,Football,NY,99.0,0.0
47,2014,Cornell University,IVY,Football,NY,97.0,0.0
48,2014,Dartmouth College,IVY,Football,NH,100.0,0.0
76,2014,Harvard University,IVY,Football,MA,98.0,0.0
162,2014,University of Pennsylvania,IVY,Football,PA,91.0,90.0
166,2014,Princeton University,IVY,Football,NJ,99.0,100.0
244,2014,Yale University,IVY,Football,CT,98.0,0.0


In [22]:
## Align all team names ##
recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace('UCF', 'Central Florida', regex=False)
recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace('Miami (FL)', 'Miami (Fla.)', regex=False)
recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace(['UTSA', 'UTEP', 'Southern Miss'], ['Texas-San Antonio', 'Texas-El Paso', 'Southern Mississippi'], regex=False)

recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace(['App State',  'Arizona St.',  'Arkansas St.', 'Army West Point', 'Ball St.'], 
                                                  ['Appalachian State',  'Arizona State',  'Arkansas State', 'Army', 'Ball State' ], regex=False)

recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace(['Boise St.',  'BYU',  'Central Mich.', 'Colorado St.',  'Eastern Mich.',  'FIU',  'Fla. Atlantic'],
                                                  ['Boise State',  'Brigham Young',  'Central Michigan', 'Colorado State',  'Eastern Michigan',  'Florida International',  'Florida Atlantic'], 
                                                  regex=False)

recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace(['Florida St.',  'Fresno St.',  'Ga. Southern', 'Georgia St.',  'Iowa St.',  'Kansas St.',  'Kent St.'],
                                                  ['Florida State',  'Fresno State',  'Georgia Southern', 'Georgia State',  'Iowa State',  'Kansas State',  'Kent State'], regex=False)

recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace(['Louisiana',  'Miami (OH)',  'Michigan St.', 'Middle Tenn.',  'Mississippi St.',  'NC State',  'New Mexico St.'],
                                                  ['Louisiana-Lafayette',  'Miami (Ohio)',  'Michigan State', 'Middle Tennessee',  'Mississippi State',  'North Carolina State',  'New Mexico State'], 
                                                  regex=False)

recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace(['NIU',  'Ohio St.',  'Oklahoma St.', 'Ole Miss',  'Oregon St.',  'Penn St.',  'San Diego St.'],
                                                  ['Northern Illinois',  'Ohio State',  'Oklahoma State', 'Mississippi',  'Oregon State',  'Penn State',  'San Diego State'], regex=False)


recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace(['San Jose St.',  'South Fla.',  'Southern Miss.', 'TCU',  'Texas St.',  'UAB',  'UConn'],
                                                  ['San Jose State',  'South Florida',  'Southern Methodist', 'Texas Christian',  'Texas State',  'Alabama at Birmingham',  'Connecticut'], 
                                                  regex=False)

recordsdf['SCHOOL'] = recordsdf['SCHOOL'].replace(['ULM',  'UNLV',  'Utah St.', 'Washington St.',  'Western Ky.',  'Western Mich.'],
                                                  ['Louisiana-Monroe',  'Nevada-Las Vegas',  'Utah State', 'Washington State',  'Western Kentucky',  'Western Michigan'], regex=False)

stadiumdf['College'] = stadiumdf['College'].replace(['USC',  'UAB',  'Miami',  'UTSA',  'BYU',  'NC State',  'UTEP',  'TCU',  'UNLV',  'Southern Miss',  'SMU',  'Miami-OH',  'UMass',  'UNC Charlotte'],
                                                  ['Southern California',  'Alabama at Birmingham',  'Miami (Fla.)',  'Texas-San Antonio',  'Brigham Young',  'North Carolina State',  'Texas-El Paso',  
                                                   'Texas Christian', 'Nevada-Las Vegas',  'Southern Mississippi',  'Southern Methodist',  'Miami (Ohio)',  'Massachusetts',  'Charlotte' ], regex=False)


graddf['School'] = graddf['School'].replace(['U.S. Air Force Academy',  'University of Akron',  'University of Alabama',  'University of Alabama at Birmingham',  'Appalachian State University',  'University of Arizona',  
                                               'Arizona State University', 'University of Arkansas, Fayetteville',  'Arkansas State University',  'U.S. Military Academy',  'Auburn University',  'Ball State University',  'Baylor University',  'Boise State University',  'Boston College', 
                                               'Bowling Green State University',  'Brigham Young University',  'University at Buffalo, the State University of New York',  'University of California, Berkeley',  'University of Central Florida',  'Central Michigan University',  
                                               'The University of North Carolina at Charlotte',  'University of Cincinnati',  'Clemson University',  'Coastal Carolina University',  'University of Colorado, Boulder',  'Colorado State University',  'University of Connecticut', 
                                               'Duke University',  'East Carolina University',  'Eastern Michigan University',  'University of Florida',  'Florida Atlantic University',  'Florida International University',  'Florida State University'],
                                                ['Air Force',  'Akron',  'Alabama',  'Alabama at Birmingham',  'Appalachian State',  'Arizona',  'Arizona State',  'Arkansas',  'Arkansas State',  'Army',  'Auburn',  'Ball State',  'Baylor',  'Boise State',  'Boston College',  
                                                 'Bowling Green',  'Brigham Young',  'Buffalo',  'California',  'Central Florida',  'Central Michigan',  'Charlotte',  'Cincinnati',  'Clemson',  'Coastal Carolina',  'Colorado',  'Colorado State',  'Connecticut',  'Duke',  'East Carolina', 
                                                 'Eastern Michigan',  'Florida',  'Florida Atlantic',  'Florida International',  'Florida State'], regex=False)


graddf['School'] = graddf['School'].replace(['California State University, Fresno',  'University of Georgia',  'Georgia Southern University',  'Georgia State University',  'Georgia Institute of Technology',  'University of Hawaii, Manoa', 
                                               'University of Houston',  'University of Illinois Urbana-Champaign',  'Indiana University, Bloomington',  'University of Iowa',  'Iowa State University',  'University of Kansas',  'Kansas State University', 
                                               'Kent State University',  'University of Kentucky',  'Liberty University',  'Louisiana Tech University',  'University of Louisiana at Lafayette',  'University of Louisiana Monroe',  'University of Louisville', 
                                               'Louisiana State University',  'Marshall University',  'University of Maryland, College Park',  'University of Massachusetts, Amherst',  'University of Memphis',  'University of Miami (Florida)',  'Miami University (Ohio)', 
                                               'University of Michigan',  'Michigan State University',  'Middle Tennessee State University',  'University of Minnesota, Twin Cities',  'University of Mississippi',  'Mississippi State University', 
                                               'University of Missouri, Columbia',  'U.S. Naval Academy'],
                                                ['Fresno State',  'Georgia',  'Georgia Southern',  'Georgia State',  'Georgia Tech',  'Hawaii',  'Houston',  'Illinois',  'Indiana',  'Iowa',  'Iowa State',  'Kansas',  'Kansas State',  'Kent State',  'Kentucky', 
                                                 'Liberty',  'Louisiana Tech',  'Louisiana-Lafayette',  'Louisiana-Monroe',  'Louisville',  'LSU',  'Marshall',  'Maryland',  'Massachusetts',  'Memphis',  'Miami (Fla.)',  'Miami (Ohio)',  'Michigan',  'Michigan State', 
                                                 'Middle Tennessee',  'Minnesota',  'Mississippi',  'Mississippi State',  'Missouri',  'Navy'], regex=False)


graddf['School'] = graddf['School'].replace(['University of Nebraska, Lincoln',  'University of Nevada, Reno',  'University of Nevada, Las Vegas',  'University of New Mexico',  'New Mexico State University',  'University of North Carolina, Chapel Hill',  'North Carolina State University',  
                                               'University of North Texas',  'Northern Illinois University',  'Northwestern University',  'University of Notre Dame',  'Ohio University',  'The Ohio State University',  'University of Oklahoma',  'Oklahoma State University',  'Old Dominion University', 
                                               'University of Oregon',  'Oregon State University',  'Pennsylvania State University',  'University of Pittsburgh',  'Purdue University',  'Rice University',  'Rutgers, The State University of New Jersey, New Brunswick',  'San Diego State University', 
                                               'San Jose State University',  'University of South Alabama',  'University of South Carolina, Columbia',  'University of South Florida',  'University of Southern California',  'Southern Methodist University',  'The University of Southern Mississippi', 
                                               'Stanford University',  'Syracuse University',  'University of Tennessee, Knoxville',  'University of Texas at Austin'],
                                                ['Nebraska',  'Nevada',  'Nevada-Las Vegas',  'New Mexico',  'New Mexico State',  'North Carolina',  'North Carolina State',  'North Texas',  'Northern Illinois',  'Northwestern',  'Notre Dame',  'Ohio',  'Ohio State',  'Oklahoma',  'Oklahoma State', 
                                                 'Old Dominion',  'Oregon',  'Oregon State',  'Penn State',  'Pittsburgh',  'Purdue',  'Rice',  'Rutgers',  'San Diego State',  'San Jose State',  'South Alabama',  'South Carolina',  'South Florida',  'Southern California',  'Southern Methodist', 
                                                 'Southern Mississippi',  'Stanford',  'Syracuse',  'Tennessee',  'Texas'], regex=False)


graddf['School'] = graddf['School'].replace(['Texas A&M University, College Station',  'Texas Christian University',  'Texas State University',  'Texas Tech University',  'University of Texas at El Paso',  'University of Texas at San Antonio',  'University of Toledo',  'Troy University',  'Tulane University', 
                                               'The University of Tulsa',  'University of California, Los Angeles',  'University of Utah',  'Utah State University',  'Vanderbilt University',  'Virginia Polytechnic Institute and State University',  'University of Virginia',  'Wake Forest University',  
                                               'University of Washington',  'Washington State University',  'West Virginia University',  'Western Kentucky University',  'Western Michigan University',  'University of Wisconsin-Madison',  'University of Wyoming'],
                                                ['Texas A&M',  'Texas Christian',  'Texas State',  'Texas Tech',  'Texas-El Paso',  'Texas-San Antonio',  'Toledo',  'Troy',  'Tulane',  'Tulsa',  'UCLA',  'Utah',  'Utah State',  'Vanderbilt',  'Virginia',  'Virginia Tech',  'Wake Forest',  'Washington',  
                                                 'Washington State',  'West Virginia',  'Western Kentucky',  'Western Michigan',  'Wisconsin',  'Wyoming'], regex=False)

Now that all Conference and School names are aligned we can start to build the dataframe.

In [23]:
## Build the dataframe, start with coaches and records ##
left_on = ['School', 'Conference']
right_on = ['SCHOOL', 'CONFERENCE']

## Merge the Dataframes ##

coach_records_df = pd.merge(coachdf, recordsdf, how='left', left_on=left_on, right_on=right_on)
coach_records_df.head()


,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout,SCHOOL,CONFERENCE,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME
0,Air Force,Mt. West,Troy Calhoun,885000.0,885000.0,247000.0,0.0,0.0,0.0,Air Force,Mt. West,10.0,3.0,403.0,257.0,31.0,19.8
1,Akron,MAC,Terry Bowden,411000.0,412500.0,225000.0,50000.0,0.0,688500.0,Akron,MAC,2.0,10.0,245.0,463.0,20.4,38.6
2,Alabama,SEC,Nick Saban,8307000.0,8307000.0,1100000.0,500000.0,0.0,33600000.0,Alabama,SEC,13.0,2.0,598.0,302.0,39.9,20.1
3,Alabama at Birmingham,C-USA,Bill Clark,900000.0,900000.0,950000.0,165471.0,0.0,3847500.0,Alabama at Birmingham,C-USA,9.0,4.0,384.0,302.0,29.5,23.2
4,Appalachian State,Sun Belt,Scott Satterfield,712500.0,712500.0,295000.0,145000.0,0.0,2160417.0,Appalachian State,Sun Belt,10.0,4.0,483.0,310.0,34.5,22.1


In [ ]:
## Drop some columns that are duplicates ## 

coach_records_df.drop(['SCHOOL', 'CONFERENCE'], axis=1, inplace=True)


In [27]:
## Check to see the columns were dropped ##

coach_records_df.head()

,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME
0,Air Force,Mt. West,Troy Calhoun,885000.0,885000.0,247000.0,0.0,0.0,0.0,10.0,3.0,403.0,257.0,31.0,19.8
1,Akron,MAC,Terry Bowden,411000.0,412500.0,225000.0,50000.0,0.0,688500.0,2.0,10.0,245.0,463.0,20.4,38.6
2,Alabama,SEC,Nick Saban,8307000.0,8307000.0,1100000.0,500000.0,0.0,33600000.0,13.0,2.0,598.0,302.0,39.9,20.1
3,Alabama at Birmingham,C-USA,Bill Clark,900000.0,900000.0,950000.0,165471.0,0.0,3847500.0,9.0,4.0,384.0,302.0,29.5,23.2
4,Appalachian State,Sun Belt,Scott Satterfield,712500.0,712500.0,295000.0,145000.0,0.0,2160417.0,10.0,4.0,483.0,310.0,34.5,22.1


In [33]:
## Rename Conference column ##

## Check to see the columns were dropped ##

coach_records_df.rename(columns={'School':'SchoolM'}, inplace=True)
coach_records_df.rename(columns={'Conference':'ConferenceM'}, inplace=True)

coach_records_df.head()


,SchoolM,ConferenceM,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME
0,Air Force,Mt. West,Troy Calhoun,885000.0,885000.0,247000.0,0.0,0.0,0.0,10.0,3.0,403.0,257.0,31.0,19.8
1,Akron,MAC,Terry Bowden,411000.0,412500.0,225000.0,50000.0,0.0,688500.0,2.0,10.0,245.0,463.0,20.4,38.6
2,Alabama,SEC,Nick Saban,8307000.0,8307000.0,1100000.0,500000.0,0.0,33600000.0,13.0,2.0,598.0,302.0,39.9,20.1
3,Alabama at Birmingham,C-USA,Bill Clark,900000.0,900000.0,950000.0,165471.0,0.0,3847500.0,9.0,4.0,384.0,302.0,29.5,23.2
4,Appalachian State,Sun Belt,Scott Satterfield,712500.0,712500.0,295000.0,145000.0,0.0,2160417.0,10.0,4.0,483.0,310.0,34.5,22.1


In [34]:
## Check to see if that worked ##

coach_records_df.head()

,SchoolM,ConferenceM,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME
0,Air Force,Mt. West,Troy Calhoun,885000.0,885000.0,247000.0,0.0,0.0,0.0,10.0,3.0,403.0,257.0,31.0,19.8
1,Akron,MAC,Terry Bowden,411000.0,412500.0,225000.0,50000.0,0.0,688500.0,2.0,10.0,245.0,463.0,20.4,38.6
2,Alabama,SEC,Nick Saban,8307000.0,8307000.0,1100000.0,500000.0,0.0,33600000.0,13.0,2.0,598.0,302.0,39.9,20.1
3,Alabama at Birmingham,C-USA,Bill Clark,900000.0,900000.0,950000.0,165471.0,0.0,3847500.0,9.0,4.0,384.0,302.0,29.5,23.2
4,Appalachian State,Sun Belt,Scott Satterfield,712500.0,712500.0,295000.0,145000.0,0.0,2160417.0,10.0,4.0,483.0,310.0,34.5,22.1


In [31]:
stadiumdf.head()

,Stadium,College,Conference,Capacity,Opened
0,Michigan Stadium,Michigan,Big Ten,107601.0,1927
1,Beaver Stadium,Penn State,Big Ten,106572.0,1960
2,Ohio Stadium,Ohio State,Big Ten,104944.0,1922
3,Kyle Field,Texas A&M,SEC,102733.0,1904
4,Neyland Stadium,Tennessee,SEC,102521.0,1921


In [43]:
## Merge the Stadium dataframe with our coaches and records df ##

## Build the dataframe, start with coaches_records and stadium ##
left_on = ['SchoolM', 'ConferenceM']
right_on = ['College', 'Conference']

## Merge the Dataframes ##

coach_records_stadium_df = pd.merge(coach_records_df, stadiumdf, how='left', left_on =left_on, right_on=right_on)
                                    
## Check that it worked ##

coach_records_stadium_df.head()



,SchoolM,ConferenceM,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME,Stadium,College,Conference,Capacity,Opened
0,Air Force,Mt. West,Troy Calhoun,885000.0,885000.0,247000.0,0.0,0.0,0.0,10.0,3.0,403.0,257.0,31.0,19.8,Falcon Stadium,Air Force,Mt. West,52237.0,1962.0
1,Akron,MAC,Terry Bowden,411000.0,412500.0,225000.0,50000.0,0.0,688500.0,2.0,10.0,245.0,463.0,20.4,38.6,InfoCision Stadium,Akron,MAC,30000.0,2009.0
2,Alabama,SEC,Nick Saban,8307000.0,8307000.0,1100000.0,500000.0,0.0,33600000.0,13.0,2.0,598.0,302.0,39.9,20.1,Bryant Denny Stadium,Alabama,SEC,101821.0,1929.0
3,Alabama at Birmingham,C-USA,Bill Clark,900000.0,900000.0,950000.0,165471.0,0.0,3847500.0,9.0,4.0,384.0,302.0,29.5,23.2,Legion Field,Alabama at Birmingham,C-USA,71594.0,1927.0
4,Appalachian State,Sun Belt,Scott Satterfield,712500.0,712500.0,295000.0,145000.0,0.0,2160417.0,10.0,4.0,483.0,310.0,34.5,22.1,Kidd Brewer Stadium,Appalachian State,Sun Belt,24150.0,1962.0


In [44]:
## Drop some columns that are duplicates ## 

coach_records_stadium_df.drop(['College', 'Conference'], axis=1, inplace=True)

coach_records_stadium_df.head()

,SchoolM,ConferenceM,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout,W,L,PF,PA,AVG_POINTS_PER _GAME,DEF_ AVG_POINTS_PER _GAME,Stadium,Capacity,Opened
0,Air Force,Mt. West,Troy Calhoun,885000.0,885000.0,247000.0,0.0,0.0,0.0,10.0,3.0,403.0,257.0,31.0,19.8,Falcon Stadium,52237.0,1962.0
1,Akron,MAC,Terry Bowden,411000.0,412500.0,225000.0,50000.0,0.0,688500.0,2.0,10.0,245.0,463.0,20.4,38.6,InfoCision Stadium,30000.0,2009.0
2,Alabama,SEC,Nick Saban,8307000.0,8307000.0,1100000.0,500000.0,0.0,33600000.0,13.0,2.0,598.0,302.0,39.9,20.1,Bryant Denny Stadium,101821.0,1929.0
3,Alabama at Birmingham,C-USA,Bill Clark,900000.0,900000.0,950000.0,165471.0,0.0,3847500.0,9.0,4.0,384.0,302.0,29.5,23.2,Legion Field,71594.0,1927.0
4,Appalachian State,Sun Belt,Scott Satterfield,712500.0,712500.0,295000.0,145000.0,0.0,2160417.0,10.0,4.0,483.0,310.0,34.5,22.1,Kidd Brewer Stadium,24150.0,1962.0
